In [1]:
import refinitiv.data as rd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go
import pandas as pd

rd.open_session()

<refinitiv.data.session.Definition object at 0x7fc228a66040 {name='codebook'}>

In [28]:

def get_maturity(t, today):
    tenor = t - today
    return tenor.days/365


def date_6M(today):
    date = today - relativedelta(months=6)
    return datetime.strftime(date, '%Y-%m-%d')


def get_data(chain='0#USBMK='):

    today = datetime.now()
    df_rics = rd.get_data(rd.discovery.Chain(chain), 'MATUR_DATE')
    rics = df_rics['Instrument'].tolist()
    Maturity = list(map(lambda t: get_maturity(t, today), df_rics.MATUR_DATE))
    df_history = rd.get_history(
        rics, fields='B_YLD_1', start=date_6M(today), count=10000).dropna()
    trace = go.Surface(y=df_history.index, x=Maturity,
                       z=df_history.values, name=chain, showscale=False)
    yield_surface = go.Figure()
    yield_surface.add_trace(trace)
    yield_surface.update_layout(template='plotly_dark', scene=dict(
        xaxis=dict(title_text='Tenor', autorange='reversed'),
        yaxis=dict(title_text='Date'),
        zaxis=dict(title_text='Yield, %')
    ),
        margin=dict(l=5, r=5, b=10, t=10),
    )
    return yield_surface

In [31]:
rd.close_session()